In [4]:
import pandas as pd
import requests
from bs4 import BeautifulSoup 
import re
import numpy as np
import requests as r
import urllib3
urllib3.disable_warnings()

In [5]:
URL='https://www.avito.ru/shops/rostov-na-donu/nedvizhimost'
req = requests.get(URL) # GET-запрос
soup = BeautifulSoup(req.text, 'lxml')


In [75]:
for link in soup.find_all('a'):
        count_pages = link.get('href')
        
num = re.findall(r'(p=\d+)', count_pages)

last = num[0].replace('p=', '')
last = int(last)

In [260]:
city = URL.replace('https://www.avito.ru/shops/', '')
city = re.findall(r'(\S+\W+)', city)
city = city[0].replace('/', '')

In [267]:
full_link = city+'?page_from=from_shops_list'

In [268]:
full_link

'rostov-na-donu?page_from=from_shops_list'

In [293]:
for_title = []
for_count = []
for_result = []
for i in range(1, last+1):
    url_link = URL+'?p=' + str(i)
    response = requests.get(url_link, verify=False)
    soup = BeautifulSoup(response.text, 'lxml')
    for_text = r.get(url_link).text
    result = re.findall(r'(\w+\S{1}\W+rostov-na-donu+\W+page_from=from_shops_list)', for_text)
    for_result.append(result)
    for row in soup.find_all('h3', attrs = {'class': 't_s_h3'}):
        for_title.append(row.text)
    for row_1 in soup.find_all('div', attrs = {'class': 't_s_items'}):
        for_count.append(row_1.text)

In [406]:
ser = pd.Series(for_result)

In [412]:
etalon = pd.DataFrame()

In [272]:
result = pd.DataFrame(data = result, columns = ['link'])
result = result.drop_duplicates(subset=['link']).reset_index()

In [273]:
links = []
for i in result['link']:
    id_avito = "https://www.avito.ru/{}".format(i)
    links.append(id_avito)

In [415]:
etalon['title'] = for_title
etalon['title'] = etalon['title'].str.replace('\n\n ', '')
etalon['title'] = etalon['title'].str.replace('\n', '')
etalon['count'] = for_count
etalon['count'] = etalon['count'].str.replace('\n\n ', '')
etalon['count'] = etalon['count'].str.replace('\n', '')
#etalon['links'] = links

In [416]:
etalon

,title,count
0,Недвижимость Ростелеком Непрофильные активы ...,1 845 объявлений
1,"АН ""Dомиан"" «Домиан» уникальная компания с у...",6 370 объявлений
2,Активы Сбербанка Продажа имущества от Сбера,1 352 объявления
3,"Банк ВТБ (ПАО) Продажа непрофильных активов,...",916 объявлений
4,Агентство недвижимости Форсаж Большой выбор ...,587 объявлений
...,...,...
197,АН Европа+ Любые операции с недвижимостью в ...,20 объявлений
198,Агенство Недвижимости,26 объявлений
199,"Апартаменты ""Наталья"" В условиях кризиса мы ...",3 объявления
200,"ЖК Военвед-парк Застройщик ЖК ""Военвед-парк""",4 объявления


In [292]:
for_text = r.get(url_link).text
result = re.findall(r'(\w+\S{1}\W+rostov-na-donu+\W+page_from=from_shops_list)', for_text)